In [352]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from transformer_lens import HookedTransformer, HookedTransformerConfig, utils
import circuitsvis as cv
import plotly.express as px
import pandas as pd
from fancy_einsum import einsum

In [353]:
# model parameter constants
N_LAYERS = 1
N_HEADS = 1
D_MODEL = 32
D_HEAD = 32
D_MLP = None
D_VOCAB = 64
SEED = 123
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [354]:

def line(tensor, line_labels=None, yaxis="", xaxis="", **kwargs):
    if isinstance(tensor, torch.Tensor):
        tensor = tensor.detach().numpy()
    
    df = pd.DataFrame({
        xaxis: np.arange(len(tensor)),
        yaxis: tensor
    })
    
    fig = px.line(df, x=xaxis, y=yaxis, labels={xaxis: xaxis, yaxis: yaxis}, **kwargs)
    
    if line_labels:
        for c, label in enumerate(line_labels):
            fig.data[c].name = label

    fig.show()

In [355]:
# data generation
import random

FIXED_LENGTH = 2

def generateLists(n, training=True):
    output = []
    for _ in range(n):
        curr = []
        for _ in range(FIXED_LENGTH):
            if training:
                curr.append(random.randint(0, D_VOCAB - 1))
            else:
                curr.append(random.randint(D_VOCAB // 2, D_VOCAB - 1))
            # curr.append(random.randint(0, 100))

        # maximum = max(curr)
        # output.append((curr, maximum))
        output.append(curr)

    output = torch.tensor(output)
    return output

In [378]:
def separate_data(n, split=0.7):
    output = []
    for i in range(D_VOCAB):
        for j in range(D_VOCAB):
            curr = [i, j]
            output.append(curr)

    random.shuffle(output)

    split_index = int(len(output) * split)
    # return training, testing
    return torch.tensor(output[:split_index]), torch.tensor(output[split_index:])

print(separate_data(2, 0.7))

(tensor([[10, 10],
        [61, 17],
        [39, 17],
        ...,
        [16,  9],
        [ 0, 13],
        [26, 35]]), tensor([[40, 54],
        [19, 35],
        [55, 44],
        ...,
        [54, 50],
        [21, 52],
        [48, 36]]))


In [357]:
def output_data(data, batch_size=128):
    for i in range(0, len(data), batch_size):
        yield data[i:i + batch_size]

In [358]:
def raw_generate():
    output = []
    for i in range(D_VOCAB):
        for j in range(D_VOCAB):
            curr = [i, j]
            output.append(curr)

    random.shuffle(output)
    return output

def cross_val_generate(output, epoch, total_epoch):
    testing_size = int(len(output) / total_epoch)
    split_index = testing_size * epoch
    # return training, testing
    testing = torch.tensor(output[split_index:split_index + testing_size])
    training = torch.tensor(output[:split_index] + output[split_index + testing_size:])
    return training, testing

In [359]:
def generateVariableLists(n, training=True):
    output = []
    for _ in range(n):
        i = 0
        j = random.randint(0, FIXED_LENGTH)
        curr = []
        while i < j:
            curr.append(random.randint(0, D_VOCAB - 1))
            i += 1
        while i < FIXED_LENGTH:
            curr.append(0)
            i += 1
        output.append(curr)

    output = torch.tensor(output)
    return output

In [360]:
# model setup
cfg = HookedTransformerConfig(
    d_model=D_MODEL,
    n_layers=N_LAYERS,
    n_heads=N_HEADS,
    d_head=D_HEAD,
    n_ctx=FIXED_LENGTH,
    d_vocab=D_VOCAB,
    act_fn="relu",
    seed=SEED,
    device=DEVICE,
    attn_only=True
)

# hooked transformer used for interpretation later
model = HookedTransformer(cfg, move_to_device=True)

In [361]:
def loss_function(logits, tokens, return_per_token=True, print_tokens=False):
    # we take the last element of the logits to make the next prediction
    logits = logits[:, -1, :]
    answer = torch.max(tokens, dim=1)[0]
    log_prob = logits.log_softmax(-1)
    if print_tokens:
        print("tokens", tokens)
        print("predicted", torch.argmax(logits, dim=-1))
    # shape is (batch_size, 1) which represents probabilities 
    # of the correct answer
    output_prob = log_prob.gather(-1, answer.unsqueeze(-1))
    if return_per_token:
        return -output_prob.squeeze()
    return -output_prob.mean()

In [362]:
def accuracy(logits, tokens, return_per_token=False):
    logits = logits[:, -1, :]
    predicted = torch.argmax(logits, dim=1)
    answer = torch.max(tokens, dim=1)[0]
    if return_per_token:
        return (predicted == answer).float()
    return (predicted == answer).float().mean().item()

In [363]:
x = torch.tensor([[1], [2], [3], [4]])

In [364]:
def train_model(model, n_epochs, batch_size, batches_per, sequence_length=2):
    lr = 1e-3
    betas = (0.9, 0.999)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, betas=betas)

    train_losses = []
    for epoch in range(n_epochs):
        epoch_losses = []
        for _ in range(batches_per):
            tokens = generateVariableLists(batch_size, training=True)
            logits = model(tokens)
            # print(tokens.shape)
            # print(logits.shape)
            losses = loss_function(logits, tokens, print_tokens=False)
            losses.mean().backward()
            optimizer.step()
            optimizer.zero_grad()
            epoch_losses.extend(losses.detach())

        train_losses.append(np.mean(epoch_losses))
        if epoch % 10 == 0:
            print(f"Epoch {epoch}, train loss: {train_losses[-1]}")

    model.eval()
    # might want to create the training and testing set beforehand
    test_data = generateVariableLists(1280, training=True)
    logits = model(test_data)
    acc = accuracy(logits, test_data, return_per_token=False)

    print(f"Test accuracy: {acc}")

    return train_losses
                

In [365]:
def train_model2(model, n_epochs, batch_size, batches_per, sequence_length=2):
    lr = 1e-3
    betas = (0.9, 0.999)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, betas=betas)

    train_losses = []
    training_data, testing_data = separate_data(batch_size)

    for epoch in range(n_epochs):
        data_generator = output_data(training_data, batch_size)
        epoch_losses = []
        for _ in range(batches_per):
            tokens = next(data_generator)
            logits = model(tokens)
            # print(tokens.shape)
            # print(logits.shape)
            losses = loss_function(logits, tokens, print_tokens=False)
            losses.mean().backward()
            optimizer.step()
            optimizer.zero_grad()
            epoch_losses.extend(losses.detach())

        train_losses.append(np.mean(epoch_losses))
        if epoch % 10 == 0:
            print(f"Epoch {epoch}, train loss: {train_losses[-1]}")

    model.eval()
    # might want to create the training and testing set beforehand
    logits = model(testing_data)
    acc = accuracy(logits, testing_data, return_per_token=False)

    print(f"Test accuracy: {acc}")

    return train_losses
                

In [366]:
def cross_val(model, n_epochs, batch_size, batches_per, sequence_length=2):
    lr = 1e-3
    betas = (0.9, 0.999)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, betas=betas)

    train_losses = []
    all_data = raw_generate()

    for epoch in range(n_epochs):
        tokens, test = cross_val_generate(all_data, epoch, n_epochs - 1)
        epoch_losses = []
        for _ in range(batches_per):
            logits = model(tokens)
            # print(tokens.shape)
            # print(logits.shape)
            losses = loss_function(logits, tokens, print_tokens=False)
            losses.mean().backward()
            optimizer.step()
            optimizer.zero_grad()
            epoch_losses.extend(losses.detach())

        train_losses.append(np.mean(epoch_losses))

        model.eval()
        logits = model(test)
        acc = accuracy(logits, test, return_per_token=False)
        if epoch % 10 == 0:
            print(f"Epoch {epoch}, train loss: {train_losses[-1]}")
            print(f"Test accuracy: {acc}")

    return train_losses
                

In [367]:
losses = train_model2(model, 100, 128, 10, 2)

Epoch 0, train loss: 4.1202392578125
Epoch 10, train loss: 0.9903205037117004
Epoch 20, train loss: 0.2248077094554901
Epoch 30, train loss: 0.08770783990621567
Epoch 40, train loss: 0.045312389731407166
Epoch 50, train loss: 0.027901560068130493
Epoch 60, train loss: 0.019090991467237473
Epoch 70, train loss: 0.013963868841528893
Epoch 80, train loss: 0.010690105147659779
Epoch 90, train loss: 0.008459190838038921
Test accuracy: 0.9585028290748596


In [368]:
line(losses, xaxis="Epoch", yaxis="Loss")

In [369]:
training_data, testing_data = separate_data(128)
train_data_gen = output_data(training_data)
tokens = next(train_data_gen)
logits, cache = model.run_with_cache(tokens)

In [370]:
attention_pattern = cache["pattern", 0, "attn"]

In [371]:
cv.attention.attention_heads(tokens=list(map(lambda t: str(t.item()), tokens[0])), attention=attention_pattern[0])

In [372]:
cv.attention.attention_patterns(tokens=list(map(lambda t: str(t.item()), tokens[0])), attention=attention_pattern[0])

In [373]:
cv.attention.attention_patterns(tokens=list(map(lambda t: str(t.item()), tokens[1])), attention=attention_pattern[1])

In [374]:
cv.attention.attention_patterns(tokens=list(map(lambda t: str(t.item()), tokens[2])), attention=attention_pattern[2])

In [375]:
cv.attention.attention_patterns(tokens=list(map(lambda t: str(t.item()), tokens[3])), attention=attention_pattern[3])

In [376]:
cv.attention.attention_patterns(tokens=list(map(lambda t: str(t.item()), tokens[4])), attention=attention_pattern[4])

In [377]:
cv.attention.attention_patterns(tokens=list(map(lambda t: str(t.item()), tokens[5])), attention=attention_pattern[5])